# 8. Convolutional Neural Networks

I recommend you take a look at these material first.

* http://www.aclweb.org/anthology/D14-1181
* https://github.com/Shawn1993/cnn-text-classification-pytorch
* http://cogcomp.org/Data/QA/QC/

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
import re
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [ ]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [ ]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [ ]:
def pad_to_batch(batch):
    x,y = zip(*batch)
    max_x = max([s.size(1) for s in x])
    x_p = []
    for i in range(len(batch)):
        if x[i].size(1) < max_x:
            x_p.append(torch.cat([x[i], Variable(LongTensor([word2index['<PAD>']] * (max_x - x[i].size(1)))).view(1, -1)], 1))
        else:
            x_p.append(x[i])
    return torch.cat(x_p), torch.cat(y).view(-1)

In [ ]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if to_index.get(w) is not None else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

## Data load & Preprocessing

### TREC question dataset(https://cogcomp.seas.upenn.edu/Data/QA/QC/)

Task involves
classifying a question into 6 question
types (whether the question is about person,
location, numeric information, etc.)

In [ ]:
import requests

def download_file(url, filename):
    """
    Download a file from a given URL

    :param url: URL to the file
    :param filename: Name of the file to save the download as
    """
    # Send a GET request to the URL
    response = requests.get(url)

    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Write the content of the response to a file
    with open(filename, 'wb') as f:
        f.write(response.content)

In [ ]:
data_file = download_file("https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label", "train_5500.label.txt")

In [ ]:
data = open('train_5500.label.txt', 'r', encoding='latin-1').readlines()

In [ ]:
data = [[' '.join(d.split(' ')[1:-1]), d.split(' ')[0].split(':')[0]] for d in data]

In [ ]:
X, y = list(zip(*data))
X = list(X)
print(X[0])
print(y[0])

How did serfdom develop in and then leave Russia
DESC


### Num masking

It reduces the search space. ex. my birthday is 12.22 ==> my birthday is ##.##

In [ ]:
for i, x in enumerate(X):
    X[i] = re.sub('\d', '#', x).split()

In [ ]:
print(X[0], y[0])

['How', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'Russia'] DESC


### Build Vocab

In [ ]:
vocab = list(set(flatten(X)))

In [ ]:
len(vocab)

9201

In [ ]:
len(set(y)) # num of class
print(set(y))

{'HUM', 'ENTY', 'DESC', 'ABBR', 'LOC', 'NUM'}


In [ ]:
word2index={'<PAD>': 0, '<UNK>': 1}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)

index2word = {v:k for k, v in word2index.items()}

target2index = {}

for cl in set(y):
    if target2index.get(cl) is None:
        target2index[cl] = len(target2index)

index2target = {v:k for k, v in target2index.items()}

In [ ]:
X_p, y_p = [], []
for pair in zip(X,y):
    X_p.append(prepare_sequence(pair[0], word2index).view(1, -1))
    y_p.append(Variable(LongTensor([target2index[pair[1]]])).view(1, -1))

data_p = list(zip(X_p, y_p))
random.shuffle(data_p)

train_data = data_p[: int(len(data_p) * 0.9)]
test_data = data_p[int(len(data_p) * 0.9):]

### Load Pretrained word vector

you can download pretrained word vector from here https://github.com/mmihaltz/word2vec-GoogleNews-vectors

## Modeling

<img src="https://raw.githubusercontent.com/DSKSD/DeepNLP-models-Pytorch/7fec64d72615933e8f4ea499c2dbaa42508f4017/images/08.cnn-for-text-architecture.png">
<center>borrowed image from http://www.aclweb.org/anthology/D14-1181</center>

In [ ]:
class  CNNClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_size, kernel_dim=100, kernel_sizes=(3, 4, 5), dropout=0.5):
        super(CNNClassifier,self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_dim, (K, embedding_dim)) for K in kernel_sizes])

        # kernal_size = (K,D)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * kernel_dim, output_size)


    def forward(self, inputs, is_training=False):
        inputs = self.embedding(inputs).unsqueeze(1) # (B,1,T,D)
        inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] #[(B,Co,W), ...]*len(Ks)
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs] #[(B,Co), ...]*len(Ks)

        concated = torch.cat(inputs, 1)

        if is_training:
            concated = self.dropout(concated) # (B,len(Ks)*Co)
        out = self.fc(concated)
        return F.log_softmax(out,1)

## Train

It takes for a while if you use just cpu.

In [ ]:
EPOCH = 5
BATCH_SIZE = 50
KERNEL_SIZES = [3,4,5]
KERNEL_DIM = 100
LR = 0.001

In [ ]:
model = CNNClassifier(len(word2index), 100, len(target2index), KERNEL_DIM, KERNEL_SIZES)

if USE_CUDA:
    model = model.cuda()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in range(EPOCH):
    losses = []
    for i,batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        inputs,targets = pad_to_batch(batch)

        model.zero_grad()
        preds = model(inputs, True)

        loss = loss_function(preds, targets)
        losses.append(loss.item())
        loss.backward()

        #for param in model.parameters():
        #    param.grad.data.clamp_(-3, 3)

        optimizer.step()

        if i % 100 == 0:
            print("[%d/%d] mean_loss : %0.2f" %(epoch, EPOCH, np.mean(losses)))
            losses = []

[0/5] mean_loss : 2.11
[1/5] mean_loss : 0.89
[2/5] mean_loss : 0.57
[3/5] mean_loss : 0.60
[4/5] mean_loss : 0.20


## Test

In [ ]:
accuracy = 0

In [ ]:
for test in test_data:
    if test[0].size(0) < 5:
        x = torch.cat([test[0], Variable(LongTensor([word2index['<PAD>']] * (5 - test[0].size(0)))).view(1, -1)], 1)
    pred = model(x).max(1)[1]
    pred = pred.item()
    target = test[1].data.tolist()[0][0]
    if pred == target:
        accuracy += 1

print(accuracy/len(test_data) * 100)

79.3040293040293


## Further topics

* <a href="https://arxiv.org/pdf/1508.06615.pdf">Character-Aware Neural Language Models</a>
* <a href="https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf">Character level CNN for text classification</a>

## Suggested Reading

* https://blog.statsbot.co/text-classifier-algorithms-in-machine-learning-acc115293278
* <a href="https://arxiv.org/pdf/1607.01759">Bag of Tricks for Efficient Text Classification</a>
* <a href="https://arxiv.org/pdf/1708.02657">Which Encoding is the Best for Text Classification in Chinese, English, Japanese and Korean?</a>